### Google Driveの読み込み

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### ライブラリの指定

In [ ]:
# プログラムの先頭において，そのプログラムで使うライブラリをimport文を用いて呼び出す
import numpy as np                # numpy: 数値計算のライブラリ
import matplotlib.pyplot as plt   # matplotlib: グラフ描画，画像表示のためのライブラリ
import matplotlib.animation as animation
from IPython import display

mydir = '/content/drive/MyDrive/SyntheticExercise/'

### 動画の確認・保存

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# 最初の図の作成：ここまでは前回と一緒
image0 = plt.imread(mydir + 'original/image0000.tif')
mappable = ax.imshow(image0, cmap='gray') # グラフを指示する関数から返り値を受け取り
cbar = fig.colorbar(mappable) # カラーバーへ入力する
cbar.set_label('Intensity') # カラーバーのラベル

# 図の中身を更新する
def update(idImage):
    img = plt.imread(f'original/image{idImage:04}.tif')
    ax.imshow(img, cmap='gray')

ani = animation.FuncAnimation(fig, update, frames=20, interval=200)

# 動画の表示
html = display.HTML(ani.to_jshtml())
display.display(html)
plt.close()

# 動画の保存
ani.save('test.mp4')

## 高度な画像処理

以下よりFRAPの回復曲線を得るための検査領域を定める．  
検査領域を定めるためには，まず領域をクロップした上で，どこの部分についての回復を見るかを決める必要がある．  
以下では条件に該当する画像の部分を指定する方法について説明する．

### 画像の二値化（binalize）

In [ ]:
cropped = image0[700:1300, 700:1300] # image[ymin:ymax, xmin:xmax] を指定すると，その範囲の配列を返す
condition1 = cropped > 500 # 条件式を満たす場合はTrue(=数字の1)，満たさない場合はFalse（=数字の0）

fig, ax = plt.subplots()
ax.imshow(condition1, cmap='gray')
plt.show()

### 二値化処理後の変数はどうなっているか？

In [ ]:
print(condition1)

### 複数条件の論理積

In [ ]:
# 輝度が1000より大きく，2000より小さい部分を抽出したい
condition1 = cropped > 1000
condition2 = cropped < 2000

fig, ax = plt.subplots()
ax.imshow(condition1*condition2, cmap='gray') # cmap: colormapをグレースケールに指定
plt.show()

### 条件で抽出した部分のデータ取得

In [ ]:
print(image0.mean()) # 画像全体の平均輝度
print(cropped.mean()) # クロップ画像の平均輝度
print(cropped[condition1].mean()) # クロップ画像のうち，condition1を満たす領域の平均輝度

### 建設中：連番処理

In [ ]:
time = []
meanIntensity = []

for idImage in range(numImage):
    imgCell0 = plt.imread(mydir + f'original/image{idImage:04}.tif').astype(np.int16)

    # meanIntensity.append(...)

### 建設中：フィッティング

In [ ]:
def func(t, a, b, l):
    return a - b*np.exp(-l*(t - 60.0))

meanIntensity = np.array(meanIntensity)
meanIntensity2 = meanIntensity[time > 50.0]
time2 = time[time > 50.0]

popt, _ = curve_fit(func, time2, meanIntensity2/meanIntensity[0], p0=[0.4, 0.2, 0.01])

print(popt)

fig, ax = plt.subplots()
ax.plot(time, meanIntensity/meanIntensity[0])
ax.plot(time2, func(time2, popt[0], popt[1], popt[2]))
plt.show()